In [1]:
# Problem:
# pos and dpos don't reset between gradient descent iterations damn




# Main resources
# https://github.com/taichi-dev/taichi/blob/master/python/taichi/examples/autodiff/minimization.py

from datetime import datetime 

start_time = datetime.now() 
import sys
sys.tracebacklimit=0


import random

import taichi as ti
import time 

# ti.init(arch=ti.gpu)
ti.init(arch=ti.cpu,cpu_max_num_threads=1,debug=True)

# Number of segments in the rope
N = 10
# Time steps the simulation goes for
T = 15

GOAL = [120,60]


u = ti.Vector.field(n=2,dtype=ti.f32, shape=())
pos = ti.Vector.field(n=2,dtype=ti.f32, shape=(T,N))
d_pos = ti.Vector.field(n=2,dtype=ti.f32, shape=(T,N))

L = ti.field(dtype=ti.f32, shape=())
ti.root.lazy_grad()
# u = ti.Vector.field(n=2,dtype=ti.f32, shape=())
# pos = ti.Vector.field(n=2,dtype=ti.f32, shape=(T,N))
# d_pos = ti.Vector.field(n=2,dtype=ti.f32, shape=(T,N))

# L = ti.field(dtype=ti.f32, shape=())

@ti.func
def F_spring(vec):  # Force of spring, see https://youtu.be/FcnvwtyxLds?t=612
    spring_k=1
    l=1
    return spring_k*( vec.norm(1e-6) - l) * vec.normalized(1e-6) 

@ti.kernel
def simulate():
#   pos[0] (initial conditions) is already done
    for t in ti.static(range(1,T)):
        for i in ti.static(range(1,N)):
#         verlet integration on all rope segments
#                 d_pos[t,i] += pos[t-1,i]-pos[t-2,i] + ti.Vector([0,10])
            d_pos[t,i] += d_pos[t-1,i] + ti.Vector([0,10])

        ######     Constraints

        
        d_pos[t,0] += (u[None] - pos[t-1,0]) * 0.25
        
        for i in ti.static(range(1,N-1)):
            v1 = F_spring(pos[t-1,i-1] - pos[t-1,i]) * 0.5
            v2 = F_spring(pos[t-1,i+1] - pos[t-1,i]) * 0.5
            d_pos[t,i] += (v1 + v2)
#             print('i =', i)
        
#         REMEMBER THE LARGER ONE (N-1) IS ON THE RIGHT
        v1 = F_spring(pos[t-1,(N-1)-1] - pos[t-1,N-1]) * 0.5
        d_pos[t,N-1] += v1
        
        for i in ti.static(range(N)):
            pos[t,i] += d_pos[t,i]

        
@ti.kernel
def compute_loss():
    for t in ti.static(range(T)):
        L[None] += t * (pos[t,N-1] - ti.Vector(GOAL)).norm(1e-6)
#     L[None] += (pos[T-1,N-1] - ti.Vector([120.0,60.0])).norm(1e-6)
        

@ti.kernel
def gradient_descent():
    for i in ti.static(range(2)):
    # Time steps the simulation goes for:
        u[None][i] -= 1 * u.grad[None][i]
#         print(u.grad[None][d])

@ti.kernel
def reset():
#     the -1 and +1 make it so that the initial conditions don't change
    for t, n in ti.ndrange(T - 1, N):
        pos[t + 1, n] = ti.Vector([0.0, 0.0])
        d_pos[t + 1 , n] = ti.Vector([0.0, 0.0])

# initialise u AFTER the ti.lazygrad or it doesn't work
u[None] = [200,-200]
def main():
    # Initialize positions of rope segments
    for i in range(N):
#         pos[0,i] = [i * random.random() * 100,i * random.random() * 100]
        pos[0,i] = [i * 10,i * 10]
        d_pos[0,i] = [0,0]

    
    # Optimize with m gradient descent iterations
    for k in range(10000):
        with ti.ad.Tape(loss=L):
            reset()
            simulate()
            compute_loss()
            print('Loss =', L[None])
        gradient_descent()
        
        u[None][0] = sorted((-500, u[None][0], 500))[1]
        u[None][1] = sorted((-500, u[None][1], 500))[1]

        
        
    time_elapsed = datetime.now() - start_time 
    print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))


if __name__ == '__main__':
    main()
    


[Taichi] version 1.3.0, llvm 15.0.1, commit 0f25b95e, win, python 3.9.12
[Taichi] Starting on arch=x64
Loss = 13782.7451171875
Loss = 13782.744140625
Loss = 13782.7421875
Loss = 13782.7421875
Loss = 13782.7412109375
Loss = 13782.7392578125
Loss = 13782.7373046875
Loss = 13782.736328125
Loss = 13782.7353515625
Loss = 13782.734375
Loss = 13782.7333984375
Loss = 13782.732421875
Loss = 13782.7314453125
Loss = 13782.73046875
Loss = 13782.7294921875
Loss = 13782.728515625
Loss = 13782.7265625
Loss = 13782.7265625
Loss = 13782.724609375
Loss = 13782.7236328125
Loss = 13782.72265625
Loss = 13782.7216796875
Loss = 13782.720703125
Loss = 13782.720703125
Loss = 13782.7197265625
Loss = 13782.71875
Loss = 13782.716796875
Loss = 13782.716796875
Loss = 13782.7158203125
Loss = 13782.7138671875
Loss = 13782.712890625
Loss = 13782.7119140625
Loss = 13782.7109375
Loss = 13782.7099609375
Loss = 13782.7080078125
Loss = 13782.70703125
Loss = 13782.7060546875
Loss = 13782.705078125
Loss = 13782.703125
Loss =

Loss = 13782.291015625
Loss = 13782.2900390625
Loss = 13782.2880859375
Loss = 13782.287109375
Loss = 13782.28515625
Loss = 13782.28515625
Loss = 13782.283203125
Loss = 13782.2822265625
Loss = 13782.28125
Loss = 13782.2802734375
Loss = 13782.2783203125
Loss = 13782.2763671875
Loss = 13782.275390625
Loss = 13782.2744140625
Loss = 13782.2734375
Loss = 13782.271484375
Loss = 13782.2705078125
Loss = 13782.26953125
Loss = 13782.267578125
Loss = 13782.2666015625
Loss = 13782.265625
Loss = 13782.263671875
Loss = 13782.2626953125
Loss = 13782.26171875
Loss = 13782.2607421875
Loss = 13782.259765625
Loss = 13782.2578125
Loss = 13782.255859375
Loss = 13782.2548828125
Loss = 13782.2529296875
Loss = 13782.251953125
Loss = 13782.2509765625
Loss = 13782.25
Loss = 13782.248046875
Loss = 13782.2470703125
Loss = 13782.24609375
Loss = 13782.2451171875
Loss = 13782.2431640625
Loss = 13782.2421875
Loss = 13782.2421875
Loss = 13782.240234375
Loss = 13782.23828125
Loss = 13782.2373046875
Loss = 13782.23632812

Loss = 13781.7529296875
Loss = 13781.751953125
Loss = 13781.75
Loss = 13781.748046875
Loss = 13781.7470703125
Loss = 13781.74609375
Loss = 13781.7431640625
Loss = 13781.7421875
Loss = 13781.7392578125
Loss = 13781.73828125
Loss = 13781.7373046875
Loss = 13781.736328125
Loss = 13781.7333984375
Loss = 13781.732421875
Loss = 13781.7314453125
Loss = 13781.73046875
Loss = 13781.728515625
Loss = 13781.7275390625
Loss = 13781.7255859375
Loss = 13781.7236328125
Loss = 13781.72265625
Loss = 13781.720703125
Loss = 13781.71875
Loss = 13781.7177734375
Loss = 13781.7158203125
Loss = 13781.71484375
Loss = 13781.712890625
Loss = 13781.7119140625
Loss = 13781.7099609375
Loss = 13781.7080078125
Loss = 13781.70703125
Loss = 13781.70703125
Loss = 13781.705078125
Loss = 13781.703125
Loss = 13781.7021484375
Loss = 13781.69921875
Loss = 13781.6982421875
Loss = 13781.697265625
Loss = 13781.6953125
Loss = 13781.693359375
Loss = 13781.693359375
Loss = 13781.6904296875
Loss = 13781.689453125
Loss = 13781.688476

Loss = 13781.177734375
Loss = 13781.1767578125
Loss = 13781.173828125
Loss = 13781.1728515625
Loss = 13781.171875
Loss = 13781.169921875
Loss = 13781.16796875
Loss = 13781.166015625
Loss = 13781.1640625
Loss = 13781.1630859375
Loss = 13781.1611328125
Loss = 13781.1591796875
Loss = 13781.1572265625
Loss = 13781.15625
Loss = 13781.154296875
Loss = 13781.15234375
Loss = 13781.1494140625
Loss = 13781.1484375
Loss = 13781.146484375
Loss = 13781.1455078125
Loss = 13781.1435546875
Loss = 13781.1416015625
Loss = 13781.140625
Loss = 13781.138671875
Loss = 13781.1357421875
Loss = 13781.1337890625
Loss = 13781.1328125
Loss = 13781.130859375
Loss = 13781.1298828125
Loss = 13781.126953125
Loss = 13781.126953125
Loss = 13781.1240234375
Loss = 13781.123046875
Loss = 13781.12109375
Loss = 13781.119140625
Loss = 13781.1171875
Loss = 13781.1162109375
Loss = 13781.11328125
Loss = 13781.11328125
Loss = 13781.109375
Loss = 13781.1083984375
Loss = 13781.1064453125
Loss = 13781.10546875
Loss = 13781.10351562

Loss = 13780.4423828125
Loss = 13780.4375
Loss = 13780.4365234375
Loss = 13780.435546875
Loss = 13780.43359375
Loss = 13780.431640625
Loss = 13780.4287109375
Loss = 13780.427734375
Loss = 13780.42578125
Loss = 13780.4228515625
Loss = 13780.421875
Loss = 13780.419921875
Loss = 13780.4169921875
Loss = 13780.4140625
Loss = 13780.4130859375
Loss = 13780.41015625
Loss = 13780.4091796875
Loss = 13780.4072265625
Loss = 13780.404296875
Loss = 13780.4033203125
Loss = 13780.4013671875
Loss = 13780.3994140625
Loss = 13780.396484375
Loss = 13780.3955078125
Loss = 13780.39453125
Loss = 13780.3896484375
Loss = 13780.388671875
Loss = 13780.38671875
Loss = 13780.384765625
Loss = 13780.3837890625
Loss = 13780.380859375
Loss = 13780.37890625
Loss = 13780.376953125
Loss = 13780.375
Loss = 13780.373046875
Loss = 13780.3701171875
Loss = 13780.369140625
Loss = 13780.365234375
Loss = 13780.3642578125
Loss = 13780.3623046875
Loss = 13780.361328125
Loss = 13780.3564453125
Loss = 13780.35546875
Loss = 13780.354

...

Loss = 13760.267578125
Loss = 13760.2666015625
Loss = 13760.2646484375
Loss = 13760.263671875
Loss = 13760.26171875
Loss = 13760.2607421875
Loss = 13760.2587890625
Loss = 13760.2587890625
Loss = 13760.2578125
Loss = 13760.2568359375
Loss = 13760.25390625
Loss = 13760.2529296875
Loss = 13760.251953125
Loss = 13760.25
Loss = 13760.2490234375
Loss = 13760.2470703125
Loss = 13760.2451171875
Loss = 13760.244140625
Loss = 13760.2421875
Loss = 13760.2421875
Loss = 13760.240234375
Loss = 13760.23828125
Loss = 13760.2373046875
Loss = 13760.236328125
Loss = 13760.2353515625
Loss = 13760.2333984375
Loss = 13760.232421875
Loss = 13760.2314453125
Loss = 13760.2294921875
Loss = 13760.228515625
Loss = 13760.2265625
Loss = 13760.224609375
Loss = 13760.224609375
Loss = 13760.2236328125
Loss = 13760.2216796875
Loss = 13760.2197265625
Loss = 13760.2177734375
Loss = 13760.2177734375
Loss = 13760.21484375
Loss = 13760.2138671875
Loss = 13760.2109375
Loss = 13760.2109375
Loss = 13760.208984375
Loss = 13760.

Loss = 13759.7158203125
Loss = 13759.71484375
Loss = 13759.712890625
Loss = 13759.7109375
Loss = 13759.7099609375
Loss = 13759.7080078125
Loss = 13759.70703125
Loss = 13759.7060546875
Loss = 13759.705078125
Loss = 13759.703125
Loss = 13759.701171875
Loss = 13759.7001953125
Loss = 13759.69921875
Loss = 13759.6982421875
Loss = 13759.6962890625
Loss = 13759.6943359375
Loss = 13759.6923828125
Loss = 13759.69140625
Loss = 13759.6904296875
Loss = 13759.6884765625
Loss = 13759.6865234375
Loss = 13759.6845703125
Loss = 13759.68359375
Loss = 13759.681640625
Loss = 13759.6806640625
Loss = 13759.6796875
Loss = 13759.677734375
Loss = 13759.6767578125
Loss = 13759.67578125
Loss = 13759.673828125
Loss = 13759.671875
Loss = 13759.671875
Loss = 13759.6689453125
Loss = 13759.66796875
Loss = 13759.6669921875
Loss = 13759.6650390625
Loss = 13759.6640625
Loss = 13759.6630859375
Loss = 13759.6611328125
Loss = 13759.66015625
Loss = 13759.658203125
Loss = 13759.65625
Loss = 13759.6552734375
Loss = 13759.6542

Loss = 13759.140625
Loss = 13759.1396484375
Loss = 13759.1376953125
Loss = 13759.13671875
Loss = 13759.134765625
Loss = 13759.1328125
Loss = 13759.1318359375
Loss = 13759.1298828125
Loss = 13759.12890625
Loss = 13759.1279296875
Loss = 13759.1259765625
Loss = 13759.125
Loss = 13759.123046875
Loss = 13759.12109375
Loss = 13759.1201171875
Loss = 13759.119140625
Loss = 13759.1162109375
Loss = 13759.115234375
Loss = 13759.11328125
Loss = 13759.11328125
Loss = 13759.111328125
Loss = 13759.111328125
Loss = 13759.107421875
Loss = 13759.1064453125
Loss = 13759.10546875
Loss = 13759.1044921875
Loss = 13759.103515625
Loss = 13759.1015625
Loss = 13759.099609375
Loss = 13759.0986328125
Loss = 13759.0966796875
Loss = 13759.095703125
Loss = 13759.0947265625
Loss = 13759.091796875
Loss = 13759.0908203125
Loss = 13759.08984375
Loss = 13759.087890625
Loss = 13759.0869140625
Loss = 13759.0849609375
Loss = 13759.0830078125
Loss = 13759.08203125
Loss = 13759.080078125
Loss = 13759.0791015625
Loss = 13759.0

Loss = 13758.59375
Loss = 13758.5927734375
Loss = 13758.58984375
Loss = 13758.5888671875
Loss = 13758.587890625
Loss = 13758.5859375
Loss = 13758.5849609375
Loss = 13758.583984375
Loss = 13758.58203125
Loss = 13758.580078125
Loss = 13758.578125
Loss = 13758.5771484375
Loss = 13758.576171875
Loss = 13758.5732421875
Loss = 13758.572265625
Loss = 13758.5712890625
Loss = 13758.568359375
Loss = 13758.568359375
Loss = 13758.5654296875
Loss = 13758.5634765625
Loss = 13758.5625
Loss = 13758.560546875
Loss = 13758.5595703125
Loss = 13758.5576171875
Loss = 13758.5556640625
Loss = 13758.5546875
Loss = 13758.552734375
Loss = 13758.5517578125
Loss = 13758.55078125
Loss = 13758.5498046875
Loss = 13758.546875
Loss = 13758.5458984375
Loss = 13758.5439453125
Loss = 13758.54296875
Loss = 13758.541015625
Loss = 13758.5390625
Loss = 13758.537109375
Loss = 13758.5361328125
Loss = 13758.53515625
Loss = 13758.533203125
Loss = 13758.53125
Loss = 13758.5302734375
Loss = 13758.5283203125
Loss = 13758.52734375
L

Loss = 13757.962890625
Loss = 13757.9609375
Loss = 13757.9599609375
Loss = 13757.9580078125
Loss = 13757.95703125
Loss = 13757.9560546875
Loss = 13757.953125
Loss = 13757.9521484375
Loss = 13757.9501953125
Loss = 13757.94921875
Loss = 13757.9482421875
Loss = 13757.9462890625
Loss = 13757.9443359375
Loss = 13757.94140625
Loss = 13757.9404296875
Loss = 13757.939453125
Loss = 13757.9365234375
Loss = 13757.935546875
Loss = 13757.9345703125
Loss = 13757.9326171875
Loss = 13757.931640625
Loss = 13757.927734375
Loss = 13757.9267578125
Loss = 13757.9248046875
Loss = 13757.923828125
Loss = 13757.9228515625
Loss = 13757.919921875
Loss = 13757.9189453125
Loss = 13757.91796875
Loss = 13757.916015625
Loss = 13757.9140625
Time elapsed (hh:mm:ss.ms) 0:01:16.009383


In [2]:
# print(pos)
print(u)
print(u.grad[None])


[ 500.     -474.6048]
[-0.07334994  0.04029161]


In [3]:
print(d_pos)

[[[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[ 1.25000000e+02 -1.18641129e+02]
  [ 0.00000000e+00  1.00000000e+01]
  [ 0.00000000e+00  1.00000000e+01]
  [ 0.00000000e+00  1.00000000e+01]
  [ 0.00000000e+00  1.00000000e+01]
  [ 0.00000000e+00  1.00000000e+01]
  [ 0.00000000e+00  1.00000000e+01]
  [ 0.00000000e+00  1.00000000e+01]
  [ 0.00000000e+00  1.00000000e+01]
  [-4.64644623e+00  5.35355377e+00]]

 [[ 9.37500000e+01 -8.89808502e+01]
  [ 6.21515579e+01 -4.39619713e+01]
  [ 0.00000000e+00  2.00000000e+01]
  [ 0.00000000e+00  2.00000000e+01]
  [ 0.00000000e+00  2.00000000e+01]
  [ 0.00000000e+00  2.00000000e+01]
  [ 0.00000000e+00  2.00000000e+01]
  [ 0.00000000e+00  2.00

In [4]:
# print(pos)

In [5]:
import pygame
import time
import math




def rendered_pos(p):
    magnification = 1
    return [p[0]/magnification + 500,p[1]/magnification + 500]



def render(t):
    
    list_pos = [ rendered_pos(pos[t,i]) for i in range(N) ]
#     print(list_pos)
    
    window.fill((255, 255, 255))
    
    
    for pair in list(zip(list_pos, list_pos[1:])):
        pygame.draw.line(window, (0,0,0), pair[0], pair[1],width=5)
    for p in list_pos[:-1]:
        pygame.draw.circle(window, (0, 255, 0),list(p), 10, 0)
    pygame.draw.circle(window, (255, 0, 0),list(list_pos[N-1]), 10, 0)
    pygame.draw.circle(window,(0,0,255),rendered_pos(GOAL), 10, 0)
    pygame.draw.circle(window,(0,255,255),rendered_pos(list(u[None])), 10, 0)
    
#     pygame.draw.circle(window, (255, 0, 0),list(anchor), 5, 0)
    # Draws the surface object to the screen.
    img = font.render(str(t), True,(0, 0, 0))
    window.blit(img, (20, 20))
    
    pygame.display.update()

pygame.init()
font = pygame.font.SysFont(None, 40)
window = pygame.display.set_mode((1000, 1000))
run = True
while run:
    for t in range(T):
#         print(t)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        render(t)
        time.sleep(0.0167 * 5)

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 